<a href="https://colab.research.google.com/github/azamatarmanuly99/docs/blob/main/notebooks/06.03-Design-with-a-resfile-and-relax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--NOTEBOOK_HEADER-->
*This notebook contains material from [PyRosetta](https://RosettaCommons.github.io/PyRosetta.notebooks);
content is available [on Github](https://github.com/RosettaCommons/PyRosetta.notebooks.git).*

<!--NAVIGATION-->
< [Packing and Relax](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.02-Packing-design-and-regional-relax.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [Protein Design 2](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.04-Protein-Design-2.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.03-Design-with-a-resfile-and-relax.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>

# Protein Design with a Resfile and FastRelax

Keywords: FastDesign, FastRelax, ResfileCommandOperation, Resfile, ResidueSelector, MoveMapFactory, TaskFactory, TaskOperation, NoRepackDisulfides, IncludeCurrent, ReadResfile, conf2pdb_chain(), pose_from_rcsb(), create_score_function(), CA_rmsd()

## Overview

In this Workshop, we will learn the classic way to design proteins, but in the same breath introduce the concept of design using a flexible backbone protocol.  

This protocol, is essentially design during FastRelax.  A separate class, FastDesign, has a bit more options for design, but essentially, they are the same.  

Many modern designs have used this FastDesign/RelaxedDesign protocol - including many Science papers from the Baker lab and the RosettaAntibodyDesign (RAbD) protocol that we will cover in another tutorial.

Before this workshop, you should read about the resfile syntax here: https://www.rosettacommons.org/docs/latest/rosetta_basics/file_types/resfiles

*Warning*: This notebook uses `pyrosetta.distributed.viewer` code, which runs in `jupyter notebook` and might not run if you're using `jupyterlab`.

In [1]:
!pip install pyrosettacolabsetup
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()
import pyrosetta; pyrosetta.init()


Mounted at /content/google_drive
Looking for compatible PyRosetta wheel file at google-drive/PyRosetta/colab.bin//wheels...
Found compatible wheel: /content/google_drive/MyDrive/PyRosetta/colab.bin/wheels//content/google_drive/MyDrive/PyRosetta/colab.bin/wheels/pyrosetta-2024.1+release.00b79147e63-cp310-cp310-linux_x86_64.whl


PyRosetta-4 2023 [Rosetta PyRosetta4.MinSizeRel.python310.ubuntu 2024.01+release.00b79147e63be743438188f93a3f069ca75106d6 2023-12-25T16:35:48] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python310.ubuntu r366 2024.01+release.00b79147e63 00b79147e63be743438188f93a3f069ca75106d6 http://www.pyrosetta.org 2023-12-25T16:35:48
core.init: command: PyRosetta -ex1 -ex2aro -database /usr/local/lib/python3.10/dist-packages/pyrosetta/database
basic.random.init_ra

**Make sure you are in the directory with the pdb files:**

`cd google_drive/My\ Drive/student-notebooks/`

In [2]:
import logging
logging.basicConfig(level=logging.INFO)
import pyrosetta
import pyrosetta.toolbox

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Initialize PyRosetta

In [3]:
pyrosetta.init("-ignore_unrecognized_res 1 -ex1 -ex2aro -detect_disulf 0")

PyRosetta-4 2023 [Rosetta PyRosetta4.MinSizeRel.python310.ubuntu 2024.01+release.00b79147e63be743438188f93a3f069ca75106d6 2023-12-25T16:35:48] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python310.ubuntu r366 2024.01+release.00b79147e63 00b79147e63be743438188f93a3f069ca75106d6 http://www.pyrosetta.org 2023-12-25T16:35:48
core.init: command: PyRosetta -ignore_unrecognized_res 1 -ex1 -ex2aro -detect_disulf 0 -database /usr/local/lib/python3.10/dist-packages/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=2036322932 seed_offset=0 real_seed=2036322932
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=2036322932 RG_type=mt19937


 For this tutorial, let's use the well-studied native protein crambin from PDB ID 1AB1 (http://www.rcsb.org/structure/1AB1).

 Setup the input pose and scorefunction:

In [4]:
start_pose = pyrosetta.toolbox.rcsb.pose_from_rcsb("1AB1", ATOM=True, CRYS=False)
pose = start_pose.clone()
scorefxn = pyrosetta.create_score_function("ref2015_cart.wts")

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 1.80442 seconds.
core.import_pose.import_pose: File '1AB1.clean.pdb' automatically determined to be of type PDB
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 1 in file 1AB1.clean.pdb. Best match rsd_type:  THR:NtermProteinFull
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeInterva

 Make of list of which residues are cysteine:

In [5]:
cys_res = []
for i, aa in enumerate(start_pose.sequence(), start=1):
    if aa == "C":
        cys_res.append(i)
print(cys_res)

[3, 4, 16, 26, 32, 40]


Inspect `start_pose` using the `PyMolMover` or `dump_pdb()`

## Design strategy:

Design away the cysteine residues (i.e. disulfide bonds) using a resfile, allowing all side-chains to re-pack and all backbone and side-chain torsions to minimize using the `FastRelax` mover.

 Read more about resfile file structure at https://www.rosettacommons.org/docs/latest/rosetta_basics/file_types/resfiles

To write a resfile, we need to know which chain to mutate.

We can see that the pose consists of only chain "A" by printing the `pose.pdb_info()` object:

In [6]:
print(pose.pdb_info())

PDB file name: 1AB1.clean.pdb
 Pose Range  Chain    PDB Range  |   #Residues         #Atoms

0001 -- 0046    A 0001  -- 0046  |   0046 residues;    00649 atoms
                           TOTAL |   0046 residues;    00649 atoms



 More programmatically, we could find which chains are in the `pose` using `pyrosetta.rosetta.core.pose.conf2pdb_chain(pose)` which returns a `pyrosetta.rosetta.std.map_unsigned_long_char` object which is iterable.

In [7]:
print(pyrosetta.rosetta.core.pose.conf2pdb_chain(pose))

map_unsigned_long_char{1: A}


In [8]:
for k, v in pyrosetta.rosetta.core.pose.conf2pdb_chain(pose).items():
    print(v)

A


 So we could write a resfile to disc indicating design specifications to mutate only the cysteine residues in chain "A".  Use the syntax described below, and save your resfile in this directory as `resfile`.

 https://www.rosettacommons.org/docs/latest/rosetta_basics/file_types/resfiles

In [10]:
### BEGIN SOLUTION
resfile = ".resfile"
with open(resfile, "w") as f:
    f.write("NATAA\n")
    f.write("start\n")
    for i in cys_res:
        f.write("{0} {1} ALLAAxc\n".format(i, pyrosetta.rosetta.core.pose.conf2pdb_chain(pose)[1]))
!cat {resfile}

### END SOLUTION

NATAA
start
3 A ALLAAxc
4 A ALLAAxc
16 A ALLAAxc
26 A ALLAAxc
32 A ALLAAxc
40 A ALLAAxc


Note that we don't necessarily need a resfile to use resfile commands.  We can now do this in an intuitive way through code and `ResidueSelectors` using the `ResfileCommandOperation`.  The main docs for the XML interface are available below, however the code-level interface is extremely similar.  Use the ? to get more info on this.  The operation is located in `pyrosetta.rosetta.core.pack.task.operation` as we saw this location in the previous tutorial.

https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/TaskOperations/taskoperations_pages/ResfileCommandOperation

 Now we can setup the TaskOperations for the `FastRelax` mover. These tell `FastRelax` which residues to design or repack during the packer steps in `FastRelax`.  You should be familiar with this from the previous tutorial

We use the `IncludeCurrent` to include the current rotamer of from the crystal structure during packing.

In [11]:
# The task factory accepts all the task operations
tf = pyrosetta.rosetta.core.pack.task.TaskFactory()

# These are pretty standard
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.InitializeFromCommandline())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.IncludeCurrent())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.NoRepackDisulfides())

# Include the resfile
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.ReadResfile(resfile))

# Convert the task factory into a PackerTask to take a look at it
packer_task = tf.create_task_and_apply_taskoperations(pose)
# View the PackerTask
print(packer_task)

core.pack.task: Packer task: initialize from command line()
#Packer_Task

Threads to request: ALL AVAILABLE

resid	pack?	design?	allowed_aas
1	TRUE	FALSE	THR:NtermProteinFull
2	TRUE	FALSE	THR
3	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
4	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
5	TRUE	FALSE	PRO
6	TRUE	FALSE	SER
7	TRUE	FALSE	ILE
8	TRUE	FALSE	VAL
9	TRUE	FALSE	ALA
10	TRUE	FALSE	ARG
11	TRUE	FALSE	SER
12	TRUE	FALSE	ASN
13	TRUE	FALSE	PHE
14	TRUE	FALSE	ASN
15	TRUE	FALSE	VAL
16	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
17	TRUE	FALSE	ARG
18	TRUE	FALSE	LEU
19	TRUE	FALSE	PRO
20	TRUE	FALSE	GLY
21	TRUE	FALSE	THR
22	TRUE	FALSE	SER
23	TRUE	FALSE	GLU
24	TRUE	FALSE	ALA
25	TRUE	FALSE	ILE
26	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
27	TRUE	FALSE	ALA
28	TRUE	FALSE	THR
29	TRUE	FALSE	TYR
30	TRUE	FALSE	THR
31	

 The PackerTask looks as intended!

 Now we can set up a `MoveMap` or a `MoveMapFactory` to specify which torsions are free to minimize during the minimization steps of the `FastDesign` mover

In [12]:
# Set up a MoveMapFactory
mmf = pyrosetta.rosetta.core.select.movemap.MoveMapFactory()
mmf.all_bb(setting=True)
mmf.all_bondangles(setting=True)
mmf.all_bondlengths(setting=True)
mmf.all_chi(setting=True)
mmf.all_jumps(setting=True)
mmf.set_cartesian(setting=True)

In [13]:
# Set up a MoveMap
# mm = pyrosetta.rosetta.core.kinematics.MoveMap()
# mm.set_bb(True)
# mm.set_chi(True)
# mm.set_jump(True)

# If needed, you could turn off bb and chi torsions for individual residues like this:

# vector1 of true/false for each residue in the pose
# subset_to_minimize = do_something_set.apply(pose)

# for i in range(1, pose.size() + 1):
#     if (not subset_to_minimize[i]):
#         mm.set_bb(i, False)
#         mm.set_chi(i, False)

 Because some Movers only take as input a `MoveMap`, for backwards-compatibility one could generate a `MoveMap` from a `MoveMapFactory` using the `MoveMapFactory` function `create_movemap_from_pose(pose)`

 Now let's double-check some more `pose` information to verify that we are ready for `FastRelax`:

In [14]:
display_pose = pyrosetta.rosetta.protocols.fold_from_loops.movers.DisplayPoseLabelsMover()
display_pose.tasks(tf)
display_pose.movemap_factory(mmf)
display_pose.apply(pose)

protocols.DsspMover: LEELLLHHHHHHHHHHHLLLLLHHHHHHHHLLEELLLLLLLLLLLL
protocols.fold_from_loops.DisplayPoseLabelsMover: SEQUENCE       TTCCPSIVARSNFNVCRLPGTSEAICATYTGCIIIPGATCPGDYAN
protocols.fold_from_loops.DisplayPoseLabelsMover: STRUCTURE      LEELLLHHHHHHHHHHHLLLLLHHHHHHHHLLEELLLLLLLLLLLL
protocols.fold_from_loops.DisplayPoseLabelsMover: MVMP_BB        **********************************************
protocols.fold_from_loops.DisplayPoseLabelsMover: MVMP_CHI       **********************************************
core.pack.task: Packer task: initialize from command line()
protocols.fold_from_loops.DisplayPoseLabelsMover: DESIGN         --**-----------*---------*-----*-------*------
protocols.fold_from_loops.DisplayPoseLabelsMover: REPACK         **********************************************
protocols.fold_from_loops.DisplayPoseLabelsMover: FOLDTREE       *********************************************C
protocols.fold_from_loops.DisplayPoseLabelsMover:
protocols.fold_from_loops.DisplayPoseL

 Setting up `FastRelax` prints the default `relaxscript`, showing the `ramp_repack_min` settings with the following assignments:
>ramp_repack_min [scale:fa_rep] [min_tolerance] [coord_cst_weight]

In [15]:
fr = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn_in=scorefxn, standard_repeats=1)
fr.cartesian(True)
fr.set_task_factory(tf)
fr.set_movemap_factory(mmf)
fr.min_type("lbfgs_armijo_nonmonotone") # For non-Cartesian scorefunctions, use "dfpmin_armijo_nonmonotone"

#Note that this min_type is automatically set when you set the cartesian option.
#  But it is good to be aware of this - as not all protocols will do this for you.

#fr.set_movemap(mm) # Could have optionally specified a MoveMap instead of MoveMapFactory
#fr.minimize_bond_angles(True) # If not using MoveMapFactory, could specify bond angle minimization here
#fr.minimize_bond_lengths(True) # If not using MoveMapFactory, could specify bond length minimization here

protocols.relax.RelaxScriptManager: Reading relax scripts list from database.
protocols.relax.RelaxScriptManager: Looking for MonomerRelax2019.txt
protocols.relax.RelaxScriptManager: ================== Reading script file: /usr/local/lib/python3.10/dist-packages/pyrosetta/database/sampling/relax_scripts/MonomerRelax2019.txt ==================
protocols.relax.RelaxScriptManager: repeat %%nrepeats%%
protocols.relax.RelaxScriptManager: coord_cst_weight 1.0
protocols.relax.RelaxScriptManager: scale:fa_rep 0.040
protocols.relax.RelaxScriptManager: repack
protocols.relax.RelaxScriptManager: scale:fa_rep 0.051
protocols.relax.RelaxScriptManager: min 0.01
protocols.relax.RelaxScriptManager: coord_cst_weight 0.5
protocols.relax.RelaxScriptManager: scale:fa_rep 0.265
protocols.relax.RelaxScriptManager: repack
protocols.relax.RelaxScriptManager: scale:fa_rep 0.280
protocols.relax.RelaxScriptManager: min 0.01
protocols.relax.RelaxScriptManager: coord_cst_weight 0.0
protocols.relax.RelaxScriptManag

For recommendations on setting `fr.min_type()` for the scorefunction being used, see: https://www.rosettacommons.org/docs/latest/rosetta_basics/structural_concepts/minimization-overview#recommendations

 Run Fast(Design)! Note: this takes ~1min 31s

In [16]:
### BEGIN SOLUTION
%time fr.apply(pose)
### END SOLUTION

core.energy_methods.CartesianBondedEnergy: Creating new peptide-bonded energy container (46)
basic.io.database: Database file opened: scoring/score_functions/elec_cp_reps.dat
core.scoring.elec.util: Read 40 countpair representative atoms
core.pack.dunbrack.RotamerLibrary: shapovalov_lib_fixes_enable option is true.
core.pack.dunbrack.RotamerLibrary: shapovalov_lib::shap_dun10_smooth_level of 1( aka lowest_smooth ) got activated.
core.pack.dunbrack.RotamerLibrary: Binary rotamer library selected: /usr/local/lib/python3.10/dist-packages/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin
core.pack.dunbrack.RotamerLibrary: Using Dunbrack library binary file '/usr/local/lib/python3.10/dist-packages/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin'.
core.pack.dunbrack.RotamerLibrary: Dunbrack 2010 library took 0.585612 seconds to load from binary
protocols.relax.FastRelax: CMD: repeat  2280.6  0  0  0.55
protocols.relax.FastRelax: CMD: coord_cst_weight

### Analysis

Inspect the resulting design!

 By how many Angstroms RMSD did the backbone Cα atoms move?

In [17]:
pyrosetta.rosetta.core.scoring.CA_rmsd(start_pose, pose)

0.6938614845275879

What is the delta `total_score` from `start_pose` to `pose`? Why is it large?

In [18]:
delta_total_score = scorefxn(pose) - scorefxn(start_pose)
print(delta_total_score)

core.energy_methods.CartesianBondedEnergy: Creating new peptide-bonded energy container (46)
-2385.912183259604


 What is the per-residue energy difference for each mutated position between `start_pose` and `pose`?

In [19]:
### BEGIN SOLUTION

for i in cys_res:
    pose_total_score = pyrosetta.rosetta.protocols.relax.get_per_residue_scores(pose, pyrosetta.rosetta.core.scoring.ScoreType.total_score)[i - 1]
    start_pose_total_score = pyrosetta.rosetta.protocols.relax.get_per_residue_scores(start_pose, pyrosetta.rosetta.core.scoring.ScoreType.total_score)[i - 1]
    print("The delta total_score for residue {} between start_pose and pose is {}".format(i, pose_total_score - start_pose_total_score))

### END SOLUTION

The delta total_score for residue 3 between start_pose and pose is -13.739342934429018
The delta total_score for residue 4 between start_pose and pose is -164.8289645911436
The delta total_score for residue 16 between start_pose and pose is -204.15978528941434
The delta total_score for residue 26 between start_pose and pose is -7.286949744680029
The delta total_score for residue 32 between start_pose and pose is -6.705718665840315
The delta total_score for residue 40 between start_pose and pose is -6.1238846315962245


<!--NAVIGATION-->
< [Packing and Relax](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.02-Packing-design-and-regional-relax.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [Protein Design 2](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.04-Protein-Design-2.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.03-Design-with-a-resfile-and-relax.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>